## Import and Load data file

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
nltk.download('punkt')
nltk.download('wordnet')
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [ ]:
for intent in intents.get('intents', []):
    patterns = intent.get('patterns', [])

    for pattern in patterns:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # add documents to the corpus
        documents.append((w, intent.get('tag')))

        # add to our classes list
        if intent.get('tag') not in classes:
            classes.append(intent.get('tag'))


lemmatize each word and remove duplicate words from the list

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

315 documents
122 classes ['ApologizingToBrother', 'ApologizingToDad', 'ApologizingToFriend', 'ApologizingToMom', 'ApologizingToSister', 'AskStudyQuestion', 'BookRecommendation', 'CelebratingBrotherAchievement', 'CelebratingDadAchievement', 'CelebratingFriendAchievement', 'CelebratingMomAchievement', 'CelebratingSisterAchievement', 'CheckStudyProgress', 'CheckingInWithAssistant', 'CheckingInWithBestFriend', 'CheeringUpBrother', 'CheeringUpDad', 'CheeringUpFriend', 'CheeringUpMom', 'CheeringUpSister', 'ChoosingFitnessActivities', 'ComplimentingCrush', 'ConversationalStarterRecommendation', 'CookingRecipeRecommendation', 'EndStudySession', 'ExpressingFeelingsToCrush', 'ExpressingGratitudeToAssistant', 'FitnessActivityRecommendation', 'FriendshipTalkRecommendation', 'GetStudyTips', 'GettingToKnowCrush', 'GreetingsToAssistant', 'IncorporatingNutritionIntoFitness', 'MovieRecommendation', 'MusicRecommendation', 'OvercomingFitnessPlateaus', 'PlanningActivitiesWithBestFriend', 'PlanningTogethe

## Create training and testing data

In [ ]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-4-451aefdaa425>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Create Model

In [ ]:
# Importing necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD  # Importing SGD from keras.optimizers
from tensorflow.keras.optimizers.legacy import SGD as legacy_SGD # Importing the legacy SGD from tensorflow.keras.optimizers.legacy

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains a number of neurons
# equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = legacy_SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Use the legacy SGD with the decay argument
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created")


Epoch 1/200
63/63 [==============================] - 1s 2ms/step - loss: 4.8221 - accuracy: 0.0063
Epoch 2/200
63/63 [==============================] - 0s 2ms/step - loss: 4.7962 - accuracy: 0.0127
Epoch 3/200
63/63 [==============================] - 0s 2ms/step - loss: 4.7605 - accuracy: 0.0413
Epoch 4/200
63/63 [==============================] - 0s 3ms/step - loss: 4.7457 - accuracy: 0.0286
Epoch 5/200
63/63 [==============================] - 0s 3ms/step - loss: 4.7032 - accuracy: 0.0349
Epoch 6/200
63/63 [==============================] - 0s 3ms/step - loss: 4.6578 - accuracy: 0.0349
Epoch 7/200
63/63 [==============================] - 0s 2ms/step - loss: 4.5665 - accuracy: 0.0413
Epoch 8/200
63/63 [==============================] - 0s 3ms/step - loss: 4.4923 - accuracy: 0.0476
Epoch 9/200
63/63 [==============================] - 0s 3ms/step - loss: 4.4350 - accuracy: 0.0349
Epoch 10/200
63/63 [==============================] - 0s 3ms/step - loss: 4.3383 - accuracy: 0.0540
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI Interface

In [ ]:
def send():
    try:
        msg = input("You: ")
        if msg.lower() == 'exit':
            raise KeyboardInterrupt("User exited the chat")

        if msg != '':
            res = chatbot_response(msg)
            print("Bot:", res)

    except KeyboardInterrupt as e:
        print(e)
        exit()

while True:
    send()


You: Hello
1/1 [==============================] - 0s 98ms/step
Bot: Hello! Welcome to TALK AI. How can I assist you today?
You: Language acquisition tips
1/1 [==============================] - 0s 20ms/step
Bot: Conversations are a great way to learn a language. I can provide tips on using communication for language acquisition. Interested?
You: Language acquisition tips
1/1 [==============================] - 0s 22ms/step
Bot: Language learning through communication is effective. Would you like guidance on incorporating language learning into conversations?
You: Feedback communication tips"
1/1 [==============================] - 0s 21ms/step
Bot: Constructive feedback is a valuable communication skill. Let me share tips on giving feedback that fosters growth and improvement.
You: motivational talks
1/1 [==============================] - 0s 22ms/step
Bot: Feeling demotivated? I'm here to provide motivational support through communication. What areas do you need encouragement in?


In [ ]:
base.mainloop()

1/1 [==============================] - 0s 12ms/step
